In [1]:
"""
Ce script permet de tracer l'orographie du modèle AROME sur une partie 
du Maroc ainsi que les stations du réseau synoptique
à partir d'un fichier de données ASCII sous le format : 
    Latitude - Longitude - Valeur du champ

Finalité : Tracer tout autre champ spatialement sur le Maroc

Mme Feddoul    Rhizlane    :  11/03/2022  Version originale
Mr  Lemkhenter Abderrazak  :  01/04/2022  Amélioration du script   

N.B.: En cas de conflit de packages, supprimer SVP le package PyGEOS 
Il est à noter que ce script a été testé sous python 3.6
"""

import pandas as pd
import numpy as np
import geopandas
from shapely.ops import unary_union
from shapely.geometry import Polygon
import matplotlib.pyplot as plt


"""
1- Lecture des données Elevation/Mask
"""  
data = np.loadtxt('INDTERREMER-ALT-AROME.dat')
Long = data[:,0]; Lat = data[:,1]; Elev = data[:,3]; Mask= data[:,2]

"""
Création d'un geopandas a partir des données Lat/Lon/Elev
"""
df  = pd.DataFrame({'X':Long, 'Y':Lat, 'Z':Elev})
gdf = geopandas.GeoDataFrame(
    df, geometry=geopandas.points_from_xy(df.X, df.Y), crs='epsg:4326')

"""
2- Lecture du shapeFile
"""
shapefile = 'regions/regions.shp'
geodf = geopandas.read_file(shapefile)

"""
Union des polygones pour avoir le Maroc comme un seul polygone pour pouvoir tracer le contour.
"""
polys=[]
for g in geodf.geometry:
    polys.append(g)
maroc = geopandas.GeoDataFrame(geometry=geopandas.GeoSeries(unary_union(polys)), crs="EPSG:4326")


"""
3- Lecture du fichier du réseau des stations
"""
stations=pd.read_csv('DGM-Network.csv')

def plot_use_shape_intersection(fig, ax):

    """
    Specifier ici les rectangle du sous-domaine cible en fermant le polygone
    """
    lat_point_list = [30., 36., 36., 30., 30.]
    lon_point_list = [-12., -12., 0., 0., -12.]

    poly_geom = Polygon(zip(lon_point_list, lat_point_list))
    poly = geopandas.GeoDataFrame(index=[0], crs="EPSG:4326", geometry=[poly_geom])
    
    zone = geopandas.overlay(poly, maroc, how='intersection')
    res=geopandas.overlay(gdf, zone, how='intersection')
    mar=geopandas.overlay(maroc, poly, how='intersection')
    
    cs = ax.scatter(res.X, res.Y,5, res.Z, cmap='gist_gray_r')
#    ss = ax.scatter(stations.LON, stations.LAT, 10, color='black')
    ax.set_title('Elevation (m)', fontsize=25)
    ax.set_xlabel('Longitudes (°)', fontsize=20)
    ax.set_ylabel('Latitudes (°)', fontsize=20)
    ax.tick_params(axis='both', which='major', labelsize=15)
    cbar = fig.colorbar(cs, fraction=0.1, pad=0.01)
    cbar.ax.tick_params(labelsize=15)
    zone.boundary.plot(ax=ax, color='k' )

"""
Utilisation de subplot et figsize pour paramètrer la taille de l'image
"""
fig, ax = plt.subplots(1, 1, figsize=(20,16))
plot_use_shape_intersection(fig, ax)
plt.savefig('Orography-AROME-NW-Morocco.png')
plt.show()

<Figure size 2000x1600 with 2 Axes>

In [2]:
""""
Convert the plot from png to pdf format
""""
from PIL import Image

image_1 = Image.open(r'Orography-AROME-NW-Morocco.png')
im_1 = image_1.convert('RGB')
im_1.save(r'Orography-AROME-NW-Morocco.pdf')